<a href="https://colab.research.google.com/github/lpanigrahi/LORA-PEFT/blob/main/GPT2_PEFT_LORA_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
WORKING_DIR = '/content/drive/MyDrive/workbook/GPT2-PEFT'

In [ ]:
%cd {WORKING_DIR}

/content/drive/MyDrive/workbook/GPT2-PEFT


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00


In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 10.9 MB/s eta 0:00:00


In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
!pip install lamini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 1.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import datasets

import torch
import time
import pandas as pd
from pprint import pprint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from transformers import GPT2Tokenizer, GPT2Model

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

In [ ]:
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
dataset_split = "train"
# Load dataset from the hub
dataset = load_dataset(dataset_name, split=dataset_split)
# Show dataset size
print(f"dataset size: {len(dataset)}")
# Show an example
# print(dataset[randrange(len(dataset))])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

dataset size: 18612


In [ ]:
dataset[200]['instruction']

'Create a program in Python that evaluates a postfix expression. The input to the program should be a string with the postfix expression. The output should be the evaluated value of the expression.'

In [ ]:
dataset[200]['input']

'"2 3 + 4 5 * *"'

In [ ]:
def format_instruction(sample):
     return f"""### Instruction: \
Use the Task below and the Input given to write the Response, \
which is a python programming code that can solve the following Task: \

### Task:
{sample['instruction']}

### Input:
{sample['input']}

### Response:

"""

In [ ]:
format_instruction(dataset[0])

'### Instruction: Use the Task below and the Input given to write the Response, which is a python programming code that can solve the following Task: \n### Task:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Response:\n\n'

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
tokenizer('My name is Lalatendu')

{'input_ids': [3666, 1438, 318, 36683, 265, 437, 84], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

### Prepare instructions dataset

In [ ]:
num_examples = len(dataset)
finetuning_dataset = []

for i in range(num_examples):
#   question = dataset[200]['instruction']
#   answer = examples["answer"][i]
    text_with_prompt_template = format_instruction(dataset[i])
    finetuning_dataset.append({"question": text_with_prompt_template, "answer": dataset[i]['output']})

from pprint import pprint
print("One datapoint in the finetuning dataset:")
pprint(finetuning_dataset[100])

One datapoint in the finetuning dataset:
{'answer': 'def filter_long_strings(string_list):\n'
           ' long_strings = []\n'
           ' for string in string_list:\n'
           ' if len(string) > 5:\n'
           ' long_strings.append(string)\n'
           ' return long_strings\n'
           '\n'
           "string_list = ['Python', 'is', 'Fun']\n"
           '\n'
           'long_strings = filter_long_strings(string_list)\n'
           'print(long_strings)',
 'question': '### Instruction: Use the Task below and the Input given to write '
             'the Response, which is a python programming code that can solve '
             'the following Task: \n'
             '### Task:\n'
             'Read a given list of strings and create a new list of all '
             'strings that are longer than 5 characters long.\n'
             '\n'
             '### Input:\n'
             "['Python', 'is', 'Fun']\n"
             '\n'
             '### Response:\n'
             '\n'}


In [ ]:
dataset['instruction'][100]

'Read a given list of strings and create a new list of all strings that are longer than 5 characters long.'

### Tokenizing a single example

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
# encoded_texts_longest = tokenizer(list_texts, padding=True)

In [ ]:
text = finetuning_dataset[0]['question'] + finetuning_dataset[0]['answer']

tokenized_input = tokenizer(
    text,
    return_tensors = "np",
    padding = True)

In [ ]:
tokenized_input['input_ids']

array([[21017, 46486,    25,  5765,   262, 15941,  2174,   290,   262,
        23412,  1813,   284,  3551,   262, 18261,    11,   543,   318,
          257, 21015,  8300,  2438,   326,   460,  8494,   262,  1708,
        15941,    25,   220,   198, 21017, 15941,    25,   198, 16447,
          257,  2163,   284, 15284,   262,  2160,   286,   257,  8379,
          286, 37014,    13,   198,   198, 21017, 23412,    25,   198,
           58,    16,    11,   362,    11,   513,    11,   604,    11,
          642,    60,   198,   198, 21017, 18261,    25,   198,   198,
            2, 11361,  2438,   198,  4299,  2160,    62, 43167,     7,
        43167,  2599,   198,   220,  2160,   796,   657,   198,   220,
          329,   997,   287,  8379,    25,   198,   220,   220,   220,
         2160, 15853,   997,   198,   220,  1441,  2160]])

### Tokenize the instruction dataset

In [ ]:
def tokenize_function(examples):

    text = examples['prompt']

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        1024
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors=pt,
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['prompt'], padding="max_length", truncation=True)

In [ ]:
finetuning_dataset_loaded = load_dataset(dataset_name, split=dataset_split)

In [ ]:
tokenized_dataset = finetuning_dataset_loaded.map(tokenize_function, batched=True)

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [ ]:
updated_dataset = tokenized_dataset.remove_columns(['instruction', 'input', 'output', 'prompt'])


In [ ]:
x = updated_dataset

In [ ]:
updated_dataset = x

In [ ]:
updated_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 18612
})

In [ ]:
updated_dataset = updated_dataset.add_column('lablels',updated_dataset['input_ids'])

In [ ]:
updated_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'lablels'],
    num_rows: 18612
})

In [ ]:
# updated_dataset = updated_dataset.remove_columns(['attention_mask'])

### Prepare Test/Train split

In [ ]:
split_dataset = updated_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)

In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'lablels'],
        num_rows: 16750
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'lablels'],
        num_rows: 1862
    })
})

# PyTorch Datasets & Dataloaders

In [ ]:

class GPT2Dataset(Dataset):

  def __init__(self, txt_list, gpt2_type="gpt2"):

    self.input_ids = [torch.tensor(x) for x in txt_list['input_ids']]
    self.attn_masks = [torch.tensor(x) for x in txt_list['attention_mask']]

    # for i in range(len(txt_list)):

    #   self.input_ids.append(torch.tensor(txt_list['input_ids'][0]))
    #   self.attn_masks.append(torch.tensor(txt_list['attention_mask'][0]))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
train_dataset = GPT2Dataset(split_dataset['train'])
test_dataset = GPT2Dataset(split_dataset['test'])

In [ ]:
batch_size = 2
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
next(iter(train_dataloader))

[tensor([[21106,   318,   281,  ..., 50256, 50256, 50256],
         [21106,   318,   281,  ..., 50256, 50256, 50256]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])]

# Setup PEFT/LoRA Finetuning

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [ ]:
original_model = GPT2Model.from_pretrained('gpt2', torch_dtype=torch.bfloat16)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
        r=32,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        # tune the embedding layer and prediction head
        # modules_to_save=["wte", "lm_head"]
)

In [ ]:
peft_model = get_peft_model(original_model,
                           lora_config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 1179648
all model parameters: 125619456
percentage of trainable model parameters: 0.94%


### Train PEFT Adapter

In [ ]:
output_dir = f"./peft-dialogue-summary-training-{str(int(time.time()))}"

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_steps = 1,
    max_steps= 2,
)

peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_args,
    train_dataset = split_dataset['train'],
)

In [ ]:
peft_trainer.train()

peft_model_path= "./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

TypeError: ignored

In [ ]:
type(split_dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
# Setup Training

In [ ]:
# from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner
from llama import BasicModelRunner


In [ ]:
model = BasicModelRunner('gpt2')

In [ ]:
model_name = "gpt2"

In [ ]:
dataset_name = "lamini_docs.jsonl"
dataset_path = f"/content/{dataset_name}"
use_hf = False

In [ ]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [ ]:
max_steps = 3

In [ ]:
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [ ]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [ ]:
model_flops = (
  original_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(original_model)
print("Memory footprint", original_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Linear(
          in_features=768, out_features=2304, bias=True
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.05, inplace=False)
          )
          (lora_A): ModuleDict(
            (default): Linear(in_features=768, out_features=32, bias=False)
          )
          (lora_B): ModuleDict(
            (default): Linear(in_features=32, out_features=2304, bias=False)
          )
          (lora_embedding_A): ParameterDict()
          (lora_embedding_B): ParameterDict()
        )
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   

In [ ]:
peft_model.to('cpu')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Linear(
              in_features=768, out_features=2304, bias=True
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=32, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=32, out_features=2304, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=

In [ ]:
max_steps = 3

In [ ]:
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [ ]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [ ]:
model_flops = (
  peft_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(peft_model)
print("Memory footprint", peft_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Linear(
              in_features=768, out_features=2304, bias=True
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=32, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=32, out_features=2304, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=

In [ ]:
trainer = Trainer(
    model=peft_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
)

TypeError: ignored

In [ ]:
model_flops

4238654570496